# Lab 4
## 95-791 Data Mining (Fall 2021) 
### Name: Xi Yan
#### Date: Friday, December 3rd, 2021

### Topics covered in this Lab:
- Market Basket Analysis
- Jupyter Notebooks Presentations


### Changing the author field and file name.

 (a) Change the `Name:` field on the Jupyter Notebook document by adding your own name.

 (b) Rename this file to "Lab4_F21_Mini2_YourHameHere.ipynb", where YourNameHere is changed to your own name.



### Installing and loading packages

Before you begin this Lab make sure you have installed all the required libraries. Load the libraries as indicated below.

* [nbconvert](https://nbconvert.readthedocs.io/en/latest/install.html)
* [mlxtend](http://rasbt.github.io/mlxtend/installation/)
* [yellowbrick](https://www.scikit-yb.org/en/latest/quickstart.html)
    * In previous Labs we upgraded our scikitlearn to 0.24, therefore make sure you install yellowbrick 1.3 (pip install yellowbrick==1.3). If you install another version just uninstall it and repeat the installation with 1.3.


In case you are having trouble with your paths try the following

**Windows**
* Using python in [windows](https://docs.python.org/3/using/windows.html)
* Adding PATH variables in [windows](https://www.educative.io/edpresso/how-to-add-python-to-path-variable-in-windows)
* It could also be that pip is not installed in your [conda environment](https://stackoverflow.com/questions/29055393/pip-is-not-recognized-in-anaconda-prompt)
* Look at this example of how to add python to your path and replicate for [packages](https://datatofish.com/add-python-to-windows-path/)

**Mac/Linux**
* Finding modules in [python](https://realpython.com/lessons/why-cant-python-find-my-modules/)
* Find out where your `pip` is pointing to by executing: *which pip*
* Using pip to install packages to [anaconda](https://stackoverflow.com/questions/41060382/using-pip-to-install-packages-to-anaconda-environment)
* If your terminal doesn't even recognize pip, start by: *conda install pip*
* Fixing your [PATH](https://landlab.readthedocs.io/en/master/install/correcting_paths.html)


Additional problems you might have:
* https://stackoverflow.com/questions/21201003/anaconda-not-finding-my-packages-installed-with-pip
* https://portal.biohpc.swmed.edu/media/filer_public/3e/e8/3ee83309-a745-43f8-909b-ea6263518896/guide-conda_02-09.pdf



You only need to install libraries once.  Once they're installed, you may use them by **importing** the libraries using the `import` command.  For today's lab, you'll want to run the following code

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as px
#from apyori import apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
%matplotlib inline
plt.style.use('seaborn-white')

---

## Market Basket Analysis

- Apriori algorithm from [mlxtend](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/)
- Apriori algorithm from [apyori](https://pypi.org/project/apyori/)

**1)a) Read the products and order_products datasets. Print the first 5 rows of products.**

In [3]:
#your code here
product = pd.read_csv('products.csv')
order = pd.read_csv('order_products.csv')
product.head(5)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [4]:
order.head(5)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


**1)b) Join products and order_products into a single dataframe called `basket`. You can us the `product_id` as the common columns. Then sort this joined `basket` dataframe by `order_id` and print the first 5 rows.**

In [10]:
#your code here
basket = pd.merge(product, order, on=['product_id'])
basket = basket.sort_values(by=['order_id'])
basket.head(5)

,product_id,product_name,aisle_id,department_id,order_id,add_to_cart_order,reordered
588447,22035,Organic Whole String Cheese,21,16,1,8,1
256931,10246,Organic Celery Hearts,83,4,1,3,0
277441,11109,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,1,2,1
1194893,43633,Lightly Smoked Sardines in Olive Oil,95,15,1,5,1
1302365,47209,Organic Hass Avocado,24,4,1,7,0


**1)c) Run the line below to create your transactions. Print out the first five rows of your transactions.**

In [11]:
transactions = basket.groupby("order_id")['product_name'].apply(list).reset_index(name='items')
#your code here
transactions.head(5)

,order_id,items
0,1,"[Organic Whole String Cheese, Organic Celery H..."
1,36,"[Super Greens Salad, Spring Water, Organic Gar..."
2,38,"[Green Peas, Organic Baby Arugula, Shelled Pis..."
3,96,"[Organic Raspberries, Organic Pomegranate Kern..."
4,98,"[Uncured Applewood Smoked Bacon, Black Beans, ..."


**1)d) Run the code above. Pick a random number from 50-100, and print out the equivalent transaction. What items does your transaction contain?**

In [13]:
#your code here
print(transactions.iloc[77])

order_id                                                 2853
items       [Organic Avocados 5 Count, Clementines, Organi...
Name: 77, dtype: object


--> Your answer here

The item lists are 'Organic Avocados 5 Count', 'Clementines', 'Organic Blueberries Clamshell', 'Original Hawaiian Sweet Rolls'


**2)e) Run the [TransactionEncoder](http://rasbt.github.io/mlxtend/user_guide/preprocessing/TransactionEncoder/) on your dataset and display the encoded dataframe.**

Steps:
1. Create an object of TransactionEncoder()
2. Create a variable and assign your fitted and transorformed object from 1 to it. It should look something like this:
    * your_variable = your_prev_variable.fit(your_dataset).transform(your_dataset)
    * For the previous code consider that you don't need the whole dataset, just the items.
3. Turn step 2 into a dataframe, where the content is the output from 2. Be sure to name your columns (you can get these from step 1).
4. Print out your dataframe

In [31]:
#your code here
encoder = TransactionEncoder()
enc = encoder.fit(transactions['items']).transform(transactions['items'])
enc = pd.DataFrame(enc, columns=encoder.columns_)
print(enc)

        #2 Coffee Filters  #2 Cone White Coffee Filters  \
0                   False                         False   
1                   False                         False   
2                   False                         False   
3                   False                         False   
4                   False                         False   
...                   ...                           ...   
131204              False                         False   
131205              False                         False   
131206              False                         False   
131207              False                         False   
131208              False                         False   

        #2 Mechanical Pencils  #4 Natural Brown Coffee Filters  \
0                       False                            False   
1                       False                            False   
2                       False                            False   
3                       Fal

**2)f) Before we can create association rules, we must obtain the most frequent item sets through the [apriori](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) algorithm. You might need to play around with a min_support for this dataset to determine a good threshold. Print your frequent itemsets dataset.**

Hint: apriori(your_dataset, min_support=your_threshold)

In [35]:
#your code here
ap = apriori(enc, min_support=0.01)
print(ap)

      support        itemsets
0    0.017514           (381)
1    0.011737           (541)
2    0.017163          (2000)
3    0.029480          (2289)
4    0.117980          (2641)
..        ...             ...
115  0.010281  (22465, 17475)
116  0.010685  (22496, 22465)
117  0.012492  (22496, 25128)
118  0.011729  (23736, 25128)
119  0.012728  (24656, 25128)

[120 rows x 2 columns]


**Metrics we looked at in class:**

- $support(A->C) = support(A+C)$
    * range: [0, 1]
    * Support is used to measure the abundance or frequency of items. 


- $confidence(A->C) = support(A+C) / support(A)$
    * range: [0, 1]
    * Probability of seeing item/s C in a transaction given that it also contains item/s A.


- $lift(A->C) = confidence(A->C) / support(C)$ 
    * range: [0, inf]
    * Measure of how ofter A->C happen together. If A and C are independent this score is exactly 1. 


**Other Metrics:**

- $leverage(A->C) = support(A->C) - support(A)*support(C)$
    * range: [-1, 1]


- $conviction = [1 - support(C)] / [1 - confidence(A->C)]$ 
    * range: [0, inf]

Read more about them [here](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/)

**1)g) Now you are ready to generate [association rules](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/)! Use the function below on your frequent itemsets. Use `support` as your metric. You will need to test a few thresholds. Print out the rules, sorting them by the `support`**

In [42]:
sup1 = association_rules(ap, metric='support', min_threshold=0.01)
sup1 = sup1.sort_values(by=['support'])
print(sup1)


   antecedents consequents  antecedent support  consequent support   support  \
10     (18278)      (2814)            0.045980            0.142719  0.010144   
11      (2814)     (18278)            0.142719            0.045980  0.010144   
23     (17475)     (22465)            0.062000            0.056467  0.010281   
22     (22465)     (17475)            0.056467            0.062000  0.010281   
24     (22496)     (22465)            0.074568            0.056467  0.010685   
25     (22465)     (22496)            0.056467            0.074568  0.010685   
29     (25128)     (23736)            0.083028            0.055583  0.011729   
28     (23736)     (25128)            0.055583            0.083028  0.011729   
20     (17475)     (18278)            0.062000            0.045980  0.012156   
21     (18278)     (17475)            0.045980            0.062000  0.012156   
27     (25128)     (22496)            0.083028            0.074568  0.012492   
26     (22496)     (25128)            0.

In [45]:
sup2 = association_rules(ap, metric='support', min_threshold=0.001)
sup2 = sup1.sort_values(by=['support'])
print(sup2)

   antecedents consequents  antecedent support  consequent support   support  \
10     (18278)      (2814)            0.045980            0.142719  0.010144   
11      (2814)     (18278)            0.142719            0.045980  0.010144   
23     (17475)     (22465)            0.062000            0.056467  0.010281   
22     (22465)     (17475)            0.056467            0.062000  0.010281   
24     (22496)     (22465)            0.074568            0.056467  0.010685   
25     (22465)     (22496)            0.056467            0.074568  0.010685   
29     (25128)     (23736)            0.083028            0.055583  0.011729   
28     (23736)     (25128)            0.055583            0.083028  0.011729   
20     (17475)     (18278)            0.062000            0.045980  0.012156   
21     (18278)     (17475)            0.045980            0.062000  0.012156   
27     (25128)     (22496)            0.083028            0.074568  0.012492   
26     (22496)     (25128)            0.

In [46]:
sup3 = association_rules(ap, metric='support', min_threshold=0.0001)
sup3 = sup1.sort_values(by=['support'])
print(sup3)

   antecedents consequents  antecedent support  consequent support   support  \
10     (18278)      (2814)            0.045980            0.142719  0.010144   
11      (2814)     (18278)            0.142719            0.045980  0.010144   
23     (17475)     (22465)            0.062000            0.056467  0.010281   
22     (22465)     (17475)            0.056467            0.062000  0.010281   
24     (22496)     (22465)            0.074568            0.056467  0.010685   
25     (22465)     (22496)            0.056467            0.074568  0.010685   
29     (25128)     (23736)            0.083028            0.055583  0.011729   
28     (23736)     (25128)            0.055583            0.083028  0.011729   
20     (17475)     (18278)            0.062000            0.045980  0.012156   
21     (18278)     (17475)            0.045980            0.062000  0.012156   
27     (25128)     (22496)            0.083028            0.074568  0.012492   
26     (22496)     (25128)            0.

**1)h) Repeat g) but using another metric from the [association rules](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/). We looked at a couple more in class. You will need to test a few thresholds for this metric. Print out the rules, sorting them by the metric you chose.**

In [47]:
#your code here
con1 = association_rules(ap, metric='confidence', min_threshold=0.01)
con1 = sup1.sort_values(by=['confidence'])
print(con1)

   antecedents consequents  antecedent support  consequent support   support  \
11      (2814)     (18278)            0.142719            0.045980  0.010144   
19      (2814)     (33836)            0.142719            0.049494  0.014847   
15      (2814)     (22496)            0.142719            0.074568  0.015243   
5       (2641)     (24656)            0.117980            0.042268  0.013566   
9       (2814)     (17475)            0.142719            0.062000  0.016447   
17      (2814)     (25128)            0.142719            0.083028  0.016569   
13      (2814)     (22465)            0.142719            0.056467  0.016889   
29     (25128)     (23736)            0.083028            0.055583  0.011729   
24     (22496)     (22465)            0.074568            0.056467  0.010685   
1       (2641)     (22496)            0.117980            0.074568  0.017042   
27     (25128)     (22496)            0.083028            0.074568  0.012492   
31     (25128)     (24656)            0.

In [48]:
con2 = association_rules(ap, metric='confidence', min_threshold=0.001)
con2 = sup1.sort_values(by=['confidence'])
print(con1)

   antecedents consequents  antecedent support  consequent support   support  \
11      (2814)     (18278)            0.142719            0.045980  0.010144   
19      (2814)     (33836)            0.142719            0.049494  0.014847   
15      (2814)     (22496)            0.142719            0.074568  0.015243   
5       (2641)     (24656)            0.117980            0.042268  0.013566   
9       (2814)     (17475)            0.142719            0.062000  0.016447   
17      (2814)     (25128)            0.142719            0.083028  0.016569   
13      (2814)     (22465)            0.142719            0.056467  0.016889   
29     (25128)     (23736)            0.083028            0.055583  0.011729   
24     (22496)     (22465)            0.074568            0.056467  0.010685   
1       (2641)     (22496)            0.117980            0.074568  0.017042   
27     (25128)     (22496)            0.083028            0.074568  0.012492   
31     (25128)     (24656)            0.

In [49]:
con3 = association_rules(ap, metric='confidence', min_threshold=0.001)
con3 = sup1.sort_values(by=['confidence'])
print(con3)

   antecedents consequents  antecedent support  consequent support   support  \
11      (2814)     (18278)            0.142719            0.045980  0.010144   
19      (2814)     (33836)            0.142719            0.049494  0.014847   
15      (2814)     (22496)            0.142719            0.074568  0.015243   
5       (2641)     (24656)            0.117980            0.042268  0.013566   
9       (2814)     (17475)            0.142719            0.062000  0.016447   
17      (2814)     (25128)            0.142719            0.083028  0.016569   
13      (2814)     (22465)            0.142719            0.056467  0.016889   
29     (25128)     (23736)            0.083028            0.055583  0.011729   
24     (22496)     (22465)            0.074568            0.056467  0.010685   
1       (2641)     (22496)            0.117980            0.074568  0.017042   
27     (25128)     (22496)            0.083028            0.074568  0.012492   
31     (25128)     (24656)            0.

**1)i) What are top 3 results from f? What are top 3 results from g? How different are they and why do you think they are different?**

In [50]:
sup1.head(3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
10,(18278),(2814),0.045980,0.142719,0.010144,0.220620,1.545836,0.003582,1.099953
11,(2814),(18278),0.142719,0.045980,0.010144,0.071078,1.545836,0.003582,1.027018
23,(17475),(22465),0.062000,0.056467,0.010281,0.165827,2.936692,0.006780,1.131099


In [51]:
con1.head(3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
11,(2814),(18278),0.142719,0.045980,0.010144,0.071078,1.545836,0.003582,1.027018
19,(2814),(33836),0.142719,0.049494,0.014847,0.104026,2.101819,0.007783,1.060864
15,(2814),(22496),0.142719,0.074568,0.015243,0.106803,1.432294,0.004601,1.036090


--> Your answer here

The first 3 results from these two sets based on two different metrics are also different. Using the support metrics, the set we selected are 10, 11, 23; and using the confidence metric, the set we selected are 11, 19, 15. I think this is due to the fact that the metrics for sorting these rows are different.

---

## Jupyter Notebook Presentation

We will use [nbconvert](https://nbconvert.readthedocs.io/en/latest/) for this question.

2) Follow these steps:
1. On your top menu go to View > Cell Toolbar > Slideshow
2. Now on every cell you'll have the the option of: Slide, Sub-slide, Fragment, Skip, Notes. Choose an option for each of your cell.
3. Once you are done save your file and head over to your terminal. Type the following:
    > jupyter nbconvert Lab4_F21_YourNameHere.ipynb --to slides --post serve
4. Go through your slides and see if you like them. Otherwise go back to point 2 and tweak the cells.
    * Make sure not everything is a slide
    * Pair up questions with code in the same slide
    * Make sure the transitions make sense
5. One you are done, submit this notebook along with your html file of your slides.


Additional resources:
* Flag configurations for [nbconvert](https://nbconvert.readthedocs.io/en/latest/config_options.html).
* Walkthrough of conversting jupyter notebook to [slides](https://medium.com/@mjspeck/presenting-code-using-jupyter-notebook-slides-a8a3c3b59d67)

---

### END OF LAB 4!

SyntaxError: invalid syntax (161799643.py, line 1)